In [1]:
!python3 -m pip install pystan

In [2]:
import pystan
import re
import numpy as np
import pandas as pd

In [12]:
# use your favourite football dataset, mine is from
# https://fixturedownload.com/sport/football
df2019 = pd.read_csv("epl-2020-GMTStandardTime.csv")
df2020 = pd.read_csv("epl-2020-GMTStandardTime.csv")
df = pd.concat([df2019,df2020]).dropna()

In [13]:
print(df.shape)
df.head()

(760, 7)


,Match Number,Round Number,Date,Location,Home Team,Away Team,Result
0,4,1,12/09/2020 12:30,Craven Cottage,Fulham,Arsenal,0 - 3
1,3,1,12/09/2020 15:00,Selhurst Park,Crystal Palace,Southampton,1 - 0
2,5,1,12/09/2020 17:30,Anfield,Liverpool,Leeds,4 - 3
3,8,1,12/09/2020 20:00,London Stadium,West Ham,Newcastle,0 - 2
4,7,1,13/09/2020 14:00,The Hawthorns,West Brom,Leicester,0 - 3


In [14]:
# get list of teams
teams_list = list(set(df["Home Team"].values))
teams_dict = dict(zip(teams_list,range(1,1+len(teams_list)))) # 1-indexed for STAN

In [16]:
# fixture mapped to numerical team id
fixtures = np.vstack([df["Home Team"].map(teams_dict),
                      df["Away Team"].map(teams_dict)])

In [17]:
# results of the games
results = np.vstack([df["Result"].map(lambda x: int(x[0:x.find(' -')])),
                     df["Result"].map(lambda x: int(x[(x.find('- ')+2):]))])

In [95]:
game_code = """
data {
  int<lower=0> K;                     // number of fixtures
  int<lower=0> T;                     // number of teams
  int<lower=0> results[K,2];          // results in score
  int<lower=0> fixtures[K,2];         // lookup for teams
}
parameters {
  vector<lower=0,upper=15>[T] attack;     // attack ability of each team
  vector<lower=0,upper=15>[T] defend;     // defend ability of each team
  vector<lower=0,upper=2>[T] home_attack; // attack correction at home
  vector<lower=0,upper=2>[T] home_defend; // defend correction at home
  vector<lower=0, upper=1>[T] zeroprob;   // probability of noscore, regardless of opponant
}
transformed parameters {
  vector[K] goal_home;
  vector[K] goal_away;
  for (i in 1:K){
    goal_home[i] = fmax(0.001,attack[fixtures[i,1]]-defend[fixtures[i,2]]+home_attack[fixtures[i,1]]);
    goal_away[i] = fmax(0.001,attack[fixtures[i,2]]-defend[fixtures[i,1]]-home_defend[fixtures[i,1]]);
  }
}
model {
  for (t in 1:T){
    attack[t] ~ normal(5,3);
    defend[t] ~ normal(3,3);
    home_attack[t] ~ uniform(0,2);
    home_defend[t] ~ uniform(0,2);
  }
  for (i in 1:K){
    if (results[i,1] == 0){
      target += log_sum_exp(bernoulli_lpmf(1 | zeroprob[fixtures[i,1]]),
                            bernoulli_lpmf(0 | zeroprob[fixtures[i,1]])+
                            poisson_lpmf(results[i,1]|goal_home[i]));
    } else {
      target += bernoulli_lpmf(0 | zeroprob[fixtures[i,1]])
                  + poisson_lpmf(results[i,1]|goal_home[i]);
    }
    if (results[i,2] == 0){
    target += log_sum_exp(bernoulli_lpmf(1 | zeroprob[fixtures[i,2]]),
                            bernoulli_lpmf(0 | zeroprob[fixtures[i,2]])+
                            poisson_lpmf(results[i,2]|goal_away[i]));
    } else {
      target += bernoulli_lpmf(0 | zeroprob[fixtures[i,2]])
                  + poisson_lpmf(results[i,2]|goal_away[i]);
    }
  }
}
"""

## Can try play with this toy data
# game_data = {"K": 5,
#              "T": 3,
#              "fixtures": [[1,2],[1,3],[2,3],[3,1],[2,1]],
#              "results": [[1,0],[1,2],[0,0],[0,4],[1,2]]}

game_data = {"K": results.shape[1],
             "T": len(teams_list),
             "fixtures": np.transpose(fixtures).tolist(),
             "results": np.transpose(results).tolist(),}

sm = pystan.StanModel(model_code=game_code)
fit = sm.optimizing(data=game_data, seed=42)

INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_8fb22eeae748b6a55cbe592e9063635f NOW.


In [96]:
pd.DataFrame({"Team":teams_list,
              "Away_ATT":fit["attack"],
              "Away_DEF":fit["defend"],
              "Home_ATT":fit["attack"]+fit["home_attack"],
              "Home_DEF":fit["defend"]+fit["home_defend"]
              }
             ).sort_values(by="Home_ATT",ascending=False)

,Team,Away_ATT,Away_DEF,Home_ATT,Home_DEF
6,Man City,5.422787,3.883738,5.511655,3.950589
11,Man Utd,5.282596,3.343966,5.283562,3.344195
18,Liverpool,5.268527,3.545086,5.268889,3.547932
1,Leicester,5.146229,3.323450,5.146994,3.323654
8,Spurs,5.069944,3.438714,5.087805,3.588182
12,Leeds,5.034542,2.903849,5.034925,3.304829
2,West Ham,4.933158,3.402891,4.963083,3.541775
9,Southampton,4.880260,2.606156,4.896931,3.281587
15,Aston Villa,4.723593,3.466617,4.850644,3.467214
17,Chelsea,4.841145,3.626383,4.847210,3.745452


In [97]:
# sample a game
def game(hometeam, awayteam, fit, n=1):
  ht_index = teams_dict[hometeam]-1
  at_index = teams_dict[awayteam]-1
  scores = np.unique(np.transpose(np.vstack(
      [
       np.random.poisson(
      fit["attack"][ht_index]+
      fit["home_attack"][ht_index]-
      fit["defend"][at_index],n
  ),
  np.random.poisson(
      fit["attack"][at_index]-
      fit["home_defend"][ht_index]-
      fit["defend"][ht_index],n
  )
      ]
  )),axis=0,return_counts=True)
  return {
      "most_likely_score": scores[0][np.argmax(scores[1])],
      "most_likely_prob":np.max(scores[1])/n,
      "home_win_prob":np.sum([scores[1][i] if 
                              scores[0][i][0]>scores[0][i][1] else 
                              0 for i in range(len(scores[0]))])/n,
      "draw_prob":np.sum([scores[1][i] if 
                              scores[0][i][0]==scores[0][i][1] else 
                              0 for i in range(len(scores[0]))])/n,
      "away_win_prob":np.sum([scores[1][i] if 
                              scores[0][i][0]<scores[0][i][1] else 
                              0 for i in range(len(scores[0]))])/n,
  }
  

In [98]:
gameresult = game("Everton","Spurs",fit,1000)

In [99]:
gameresult

{'away_win_prob': 0.551,
 'draw_prob': 0.214,
 'home_win_prob': 0.235,
 'most_likely_prob': 0.11,
 'most_likely_score': array([1, 2])}